In [2]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

In [3]:
### variaveis
DELAY = 1
WAIT = 30
outdir = 'C:/Users/OI336798/Documents/TCC/WS/'
file = 'meuplano.csv'
file2 = 'capitaias_meu_plano.xlsx'

In [13]:
# Definindo data para pegar os arquivos
from datetime import date,datetime,timedelta
hoje = date.today()
ds=datetime.today().isoweekday()

if ds==1:
    i = 0
    f = 5    
elif ds==2:
    i = 6
    f = 11
elif ds==3:
    i = 12
    f = 17
elif ds==4:
    i = 18
    f = 23
elif ds==5:
    i = 24
    f = 27
else:
    print('Final de semana')
    i = 0
    f = 0


In [13]:
### Importando lista de cidades_uf

df = pd.read_excel(outdir+file2)

### Coleta de dados de https://melhorplano.net/

##Criando listas
cidades=[]
data=[]
planos=[]
velocidade=[]
precos=[]

delay = 10

cidade = ''

try:
    #for n in range(len(df['cidade_uf'])):
    #for n in range(26,27):
        driver = webdriver.Chrome(executable_path=r"C:\BATS\chromedriver.exe")
        cidade = df['cidade_uf'][n]
        driver.get("https://melhorplano.net/internet-banda-larga/fibra-optica?l="+cidade)
        print('Parsing {}...'.format(cidade))
        element_present = EC.presence_of_element_located((By.XPATH, '/html/body/section[4]/div/div/div[2]/div[3]/div[2]/button'))
        WebDriverWait(driver, WAIT).until(element_present)
        
        ##Testa se existe o botão Ver Mais Resultados e clicar nele
        elem = driver.find_element(By.XPATH,'/html/body/section[4]/div/div/div[2]/div[3]/div[2]/button')
        while elem.text =='VER MAIS RESULTADOS':
            try:
                time.sleep(15)
                elem.click()
                print('click')
                elem = driver.find_element(By.XPATH,'/html/body/section[4]/div/div/div[2]/div[3]/div[2]/button')
            except:
                break
        ##Pegando data de atualização do site
        data_atualizacao =driver.find_element(By.XPATH,'/html/body/section[4]/div/div/div[2]/div[3]/div/div/span').text
        
        ### Pegando Planos, preços e velocidade
        
        ##Planos e Data de Atualização
        for i in range(1,100):
            try:
                planos.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[1]/div[2]/div[2]/div[1]/div/h3').text)
                cidades.append(cidade)
            except:
                break
        
        ## Velocidade & removendo ''
        for i in range(1,100):
            try:
                velocidade.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[2]/div[1]/div[1]/div/div[1]').text)     
                try:
                    while True:
                        velocidade.remove('')
                except ValueError:
                    pass                    
            except:
                break      

        ##Preços
        for i in range(1,100):
            try:
                precos.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[2]/div[1]/div[2]/div/div/div[1]/div/span[2]').text\
                              +driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[2]/div[1]/div[2]/div/div/div[1]/div/span[3]').text)     

            except:
                break
        
        ## Gravar dados no dataframe
        result = pd.DataFrame(planos,columns=['planos'])
        result['velocidade']=velocidade
        result['precos']=precos
        result['cidade']=cidades
        result['atualização']=data_atualizacao
        
        driver.close()
        time.sleep(delay)
        delay = delay + 10 
except:
    print('Apresentou erro')
    pass
                

#Salvar dados do DF em CSV        
result.to_csv(outdir+file, sep=';', index=False, encoding='UTF-32', mode = 'a', header = False) 



C:\Users\OI336798\AppData\Local\Temp/ipykernel_1468/3783465624.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\BATS\chromedriver.exe")


Parsing vitoria-es...
click
click


C:\Users\OI336798\AppData\Local\Temp/ipykernel_1468/3783465624.py:44: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  planos.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[1]/div[2]/div[2]/div[1]/div/h3').text)
C:\Users\OI336798\AppData\Local\Temp/ipykernel_1468/3783465624.py:52: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  velocidade.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[2]/div[1]/div[1]/div/div[1]').text)
C:\Users\OI336798\AppData\Local\Temp/ipykernel_1468/3783465624.py:64: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  precos.append(driver.find_element_by_xpath('/html/body/section[4]/div/div/div[2]/div[2]/div['+str(i)+']/div/div[2]/div[1]/div[2]/div/div